<div style="direction:rtl; text-align:right">
    
# سحب محاضر الاجتماعات الاسبوعيّة لمجلس إدارة الجهاز المركزي من الموقع الرسمي
<div style="direction:rtl; text-align:right">
يقوم هذا الملف بسحب محاضر الاجتماعات الاسبوعيّة من موقع الجهاز المركزي للمناقصات العامّة. يقوم الكود أدناه بعمل web scraping لجدول محاضر الاجتماع ثم يحفظ الجدول على شكل ملف csv  + يحفظ آخر ١٥ محضر اجتماع على شكل ملف PDF 
    
 
    
### مالهدف من هذا البرنامج؟
<div style="direction:rtl; text-align:right">
يعرض موقع الجهاز المركزي للمناقصات العامّة - مشكوراً - محاضر اجتماعاته الإسبوعية والتي يعلن من خلالها طلبات الجهات الحكومية (طلبات تمديد عقود او طلبات التعاقد المباشر) بالإضافة لقرار الجهاز المركزي على هذه الطلبات. ولكن لا يوجد ارشيف لهذه المحاضر وهذا الهدف من هذا الكود حيث يتم ارشفة وحفظ هذه المحاضر.
    
<br>
<br>
<div style="direction:rtl; text-align:right">
عمل: مطلق المطيري

### Libraries and dependencies
- Pandas
- BeautifulSoup
- Selenium
- Requests
    - **Note:** Selenium requires a driver to interface with the chosen browser. Firefox, for example, requires geckodriver, which needs to be installed before the below examples can be run.

---

# Start here

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium.webdriver.firefox.options import Options #Used so that selenium doesn't open a Firefox window
from pathlib import Path
import requests
import random
import string

In [6]:
from selenium.webdriver.firefox.options import Options #Used so that selenium doesn't open a Firefox window
Options = Options()
Options.headless = True #Yes, no Firefox window!
driver = webdriver.Firefox(options=Options)
webpage = "https://capt.gov.kw/ar/"
driver.get(webpage)

html = driver.page_source
soup = BeautifulSoup(html)

In [7]:
#Retrieve the table
inner_table = soup.findAll('table', {"class":"tenderpoptable2"}) 


# Retrieve the head of the table.
head = soup.find('thead') # Find the thead tag.
column_name = head.find_all('th') 
columns = []
for i in range(len(column_name)):
    columns.append(column_name[i].text)
columns[-1] = 'تحميل' # the 'download' column doesn't scrape the correct word, so we do that.

# Retrieve rows
rows = soup.findAll('tr')

# write the records, prepare to be added to a Pandas Dataframe.
records= []
for row in rows:
    record = []
    cells = row.findAll('td')
    for cell in cells:
        record.append(cell.text)
    records.append(record)
records[1][-1]
records.pop(0)

# Create a Dataframe and add the data to it. 
df = pd.DataFrame(records, columns=columns) #Create Dataframe
df = df.drop(list(range(15,31)))
df.set_index(df['مسلسل البند'], drop=True, inplace=True) # change the index.
df.drop(columns='مسلسل البند', inplace=True) #drop the column of the new index

# The four lines below generates a random string to avoid overwriting when using the code periodically.
abc = string.ascii_lowercase
arbitrary_string = ''.join(random.choice(abc) for i in range(4))
meetingtable_name = "meetingstable_"
tablename = meetingtable_name + arbitrary_string + ".csv"

df.to_csv(tablename) # Save to computer as a csv file.


# retrieve and prepare the meetings' pdf links for download.
records= [] # due to laziness, I'm using the some already used variable.
download_links = []
original_link = ''
for row in rows:
    links = row.findAll('a', href=True)
    for link in links:
        #print(link['href'])
        website = 'https://capt.gov.kw'
        original_link = website + link['href']
        #print(original_link)
    download_links.append(original_link)
download_links = download_links[1:16]

###### 








initial_filename = '{}.pdf'
for i in range(15):
    # File name here is also randomized to avoid overwriting.
    meetingfile_name = "CAPTmeeting_" + ''.join(random.choice(abc) for i in range(10))
    filename = Path(initial_filename.format(meetingfile_name))
    url = download_links[i]
    response = requests.get(url)
    filename.write_bytes(response.content)





For inqueries, questions about the code, feedback or suggestions, contact: Email: majhoolism@gmail.com, Twitter: @Techtology.